In [6]:
import pandas as pd
import pyodbc

import configparser

In [9]:
config_path = './db.config'
config = configparser.ConfigParser()
config.read_file(open(config_path))

server = config.get('db', 'ip')
databasename = config.get('db', 'databasename')
username = config.get('db', 'username')
password_fake = config.get('db', 'password')

print(server, databasename, username, password_fake)

172.16.99.239 DailySales bi_consulta abc


In [4]:
username = 'bi_consulta'
#password = ''

conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=172.16.99.239;'
                      'Database=DailySales;'
                      'UID='+username+';'
                      'PWD='+password+';')

cursor = conn.cursor()

In [3]:
params = ('Driver={SQL Server};'
          'Server=172.16.99.239;'
          'Database=DailySales;'
          'UID='+username+';'
          'PWD='+password+';')
        
conn = pyodbc.connect(params)
cursor = conn.cursor()

In [4]:
df = pd.read_sql_query('SELECT * FROM DailySales.dbo.Empresas',conn)

In [5]:
df.head()

,IdEmpresa,Empresa,RazonSocial,BDTalentos,Servidor,DbGP,Pais,Cono,Marca,POS,...,NitFormateo,OficinaVentasSAP,OrganizacionVentasSAP,InicialMarcaSAP,Regimen,RegimenCorto,RegimenResolucion,Moneda,SiglaMoneda,FicheroHIPOS
0,1,Iberomoda ...,IBEROMODA S.A.S,TAL_IBE,[172.16.200.254],BDIBE,Colombia,N,BERSHKA,0,...,900.207.065-3,0003,5103,B,GRAN CONTRIBUYENTE,REGIMEN COMUN,RESOLUCION 10562 DE 30 DE OCTUBRE DE 2008,Pesos,COP,None
1,2,Tendenzanova ...,TENDENZA NOVA S.A.S,TAL_TEN,[172.16.200.254],BDTEN,Colombia,N,MASSIMO DUTTI,0,...,900.232.961-3,0004,5104,M,,REGIMEN COMUN,None,Pesos,COP,None
2,3,Tematextil ...,TEMATEXTIL S.A.S,TAL_TEM,[172.16.200.254],BDTEM,Colombia,N,OYSHO,0,...,900.467.959-7,0007,5107,N,NO SOMOS GRANDES CONTRIBUYENTES,REGIMEN COMUN,None,Pesos,COP,None
3,4,Andimoda ...,ANDIMODA S.A.S,TAL_ANM,[172.16.200.254],BDANM,Colombia,N,PULL & BEAR,0,...,900.468.085-1,0005,5105,P,GRAN CONTRIBUYENTE,REGIMEN COMUN,None,Pesos,COP,None
4,5,EquiFashion ...,EQUIFASHION S.A.,TAL_EQU,[172.16.200.254\Ecuador],ECFAS,Ecuador,N,STRADIVARIUS,1,...,None,0002,5902,V,None,None,None,Dolares,USD,\\172.16.200.209\Respaldo\Stradivarius


In [ ]:
df = pd.read_sql_query('SELECT top 10000 * FROM DailySales.dbo.VDFColombia',conn)

In [6]:
len(df)

34

In [7]:
df = pd.read_sql_query('SELECT top 10 * FROM DailySales.dbo.VDFColombia',conn)

In [8]:
import urllib
import sqlalchemy

params = ('Driver={SQL Server};'
          'Server=172.16.99.239;'
          'Database=DailySales;'
          'UID='+username+';'
          'PWD='+password+';')

db_params = urllib.parse.quote_plus(params)
engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect={}".format(db_params))
#df is the dataframe; test is table name in which this dataframe is #inserted


In [9]:
df.to_sql('##tmp_test',engine,index=False,if_exists="replace",schema="dbo")

In [22]:
cursor.execute('DROP TABLE tempdb.dbo.tmp_test')

ProgrammingError: ('42S02', "[42S02] [Microsoft][ODBC SQL Server Driver][SQL Server]Cannot drop the table 'tempdb.dbo.tmp_test', because it does not exist or you do not have permission. (3701) (SQLExecDirectW)")

In [13]:
pd.read_sql_query('SELECT * FROM ##tmp_test',conn)

DatabaseError: Execution failed on sql 'SELECT * FROM ##tmp_test': ('42S02', "[42S02] [Microsoft][ODBC SQL Server Driver][SQL Server]Invalid object name '##tmp_test'. (208) (SQLExecDirectW)")

In [23]:
from sqlalchemy import event
@event.listens_for(engine, "before_cursor_execute")
def receive_before_cursor_execute(
       conn, cursor, statement, params, context, executemany
        ):
            if executemany:
                cursor.fast_executemany = True

In [24]:
df.to_sql('tempdb.dbo.tmp_test',engine,index=False,if_exists="replace",schema="dbo")

MemoryError: 